In [1]:
import requests
from bs4 import BeautifulSoup
import csv

def trade_spider(max_pages):
    page = 1
    csvfile = open('eggs.csv', 'w', newline='')
    spamwriter = csv.writer(csvfile, delimiter=';')
    
    spamwriter.writerow(['Produto', 'Quantidade','Unitario','Total','Venda Ano', 'Loja', 'Cidade'])
    
    while page <= max_pages:
        url = 'https://lista.mercadolivre.com.br/beleza-cuidado-pessoal/produtos-cabelo/aneethun_Desde_' + str(page)
        
        source_cod = requests.get(url)
        plain_text = source_cod.text
        
        soup = BeautifulSoup(plain_text, "html5lib")
        #soup = BeautifulSoup(source_cod, "html5lib")
        #links = soup.findAll('a', {'class': 'item-link item__js-link'})
        links = soup.findAll("li", {"class": "results-item article grid item-info-height-185"})
        #print(links)
        for link in links:
            #href = link.get('href')
            #price = link.findAll('span', {'class': 'main-title'})
            href = link.find('a', {'class': 'item-link item__js-link'}).get('href')
            #print(href)
            produto = link.find('span', {'class': 'main-title'}).text
            vendido = link.find('div', {'class': 'item__condition'}).text
            price = link.find('span', {'class': 'price__fraction'}).text
            priceDecimal = link.find('span', {'class': 'price__decimals'})
            if priceDecimal:
                priceDecimal = priceDecimal.text
            else:
                priceDecimal = '00';
            price = float(price+'.'+priceDecimal)
            
            amount = [int(s) for s in vendido.split() if s.isdigit()]
            if amount:
                amount = amount[0]
                total = round(price * amount , 2)
            else:
                amount = 0
                total = 0.0
                
            
        
            #print(produto + " - " + vendido)
            #nome = link.string
            
            
            linkLoja = cada_item(href)
            cidadeLoja = cidade(linkLoja)
            anos = experience(linkLoja)
            anos = [int(s) for s in anos.split() if s.isdigit()]
            if anos:
                anos = int(anos[0])
            else:
                anos = 0;
                
            if anos:
                vendaAno = round(total / anos, 2)
            else:
                vendaAno = total
            
            #print(linkLoja)
            #print(cidadeLoja)
            #spamwriter.writerow([produto, vendido, linkLoja, cidadeLoja])
            spamwriter.writerow([produto, amount,price,total,vendaAno, linkLoja, cidadeLoja])
            
        page += 48
        
    csvfile.close()


def cada_item(item_url):
    source_cod = requests.get(item_url)
    plain_text = source_cod.text
    soup = BeautifulSoup(plain_text, "html5lib")
    link = soup.find('a', {'class': 'reputation-view-more'})
    href = link.get('href')
    return href


def cidade(cidade_url):
    source_cod = requests.get(cidade_url)
    plain_text = source_cod.text
    soup = BeautifulSoup(plain_text, "html5lib")
    link = soup.find('p',{'class': 'location__description'})
    cidade = link.text
    return cidade

def experience(cidade_url):
    source_cod = requests.get(cidade_url)
    plain_text = source_cod.text
    soup = BeautifulSoup(plain_text, "html5lib")
    link = soup.find('span',{'class': 'experience'})
    cidade = link.text
    return cidade
        
        
def pegarTag(url, tag, id, selector):
    for i in range(1,15):
        
        dom = requests.get(url)
        plain_text = dom.text
        soup = BeautifulSoup(plain_text, "html5lib")
        for tag in soup.findAll(tag, {selector: id}):
            if tag.has_attr('src'):
                print(tag.get('src'))
            else:
                print('nao tem')

In [3]:
trade_spider(2000)





